In [1]:
import os
from os.path import join, exists, basename, dirname, expanduser
from glob import glob
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr
import rioxarray as rxa
import geopandas as gpd
from scipy.optimize import curve_fit
from rasterio.enums import Resampling

with open(expanduser('~/scratch/data/uavsar/image_fps'), 'rb') as f:
    image_fps = pickle.load(f)

image_fps = [f for f in image_fps if f['fp'].endswith('.unw.grd.tiff')]
netcdf_fps = '/bsuhome/zacharykeskinen/scratch/data/uavsar/images/vv_coherence'

for fp in glob(join(netcdf_fps, '*')):
    with open(fp, 'rb') as f:
        cor_vv = pickle.load(f)

In [48]:
from scipy.optimize import curve_fit

def fit_coh_decay_model(cohs, days, tau_guess, gamma_inf_guess = 0.3):
    # https://rowannicholls.github.io/python/curve_fitting/exponential.html

    # Fit the function a * np.exp(b * t) + c to x and y
    params, pcov = curve_fit(lambda t, gamma_inf, tau: gamma_inf + (1 - gamma_inf) * np.exp(- t / tau), x, y, p0=(gamma_inf_guess, tau_guess))

    gamma_inf, tau = params

    return gamma_inf, tau, pcov

In [44]:
for fp in glob(join(netcdf_fps, '*Lowman*')):
    with open(fp, 'rb') as f:
        cor_vv = pickle.load(f)

In [49]:
days = [int(d.split('_')[-1].replace('d','')) for d in cor_vv.band.values]
taus = np.apply_along_axis(arr = cor_vv['cor_vv'].values, func1d = fit_coh_decay_model, axis = 0, days = days, tau_guess = 0.3)
# da = cor_vv.isel({'band':0})['cor_vv']
# da = da.drop('band')
# da.values = taus
# da.rio.to_raster(join('/bsuhome/zacharykeskinen/uavsar/results/taus', basename(fp).replace('.pkl','.tif')))

/bsuhome/zacharykeskinen/miniconda3/envs/proj/lib/python3.9/site-packages/numpy/lib/shape_base.py:379: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  res = asanyarray(func1d(inarr_view[ind0], *args, **kwargs))
/bsuhome/zacharykeskinen/miniconda3/envs/proj/lib/python3.9/site-packages/numpy/lib/shape_base.py:402: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  buff[ind] = asanyarray(func1d(inarr_view[ind], *args, **kwargs))


In [ ]:
taus